In [0]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [6]:
from google.colab import files
uploaded = files.upload()

Saving movies.csv to movies.csv


In [7]:
from google.colab import files
uploaded = files.upload()

Saving ratings.csv to ratings.csv


In [0]:
#import movie csv with description, and more than 10 user ratings
movies = pd.read_csv('movies.csv', ',')#, index_col=0, names=['title', 'genres'], skiprows=1)
ratings = pd.read_csv('ratings.csv', sep =',')

In [9]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
def extractor(line):
    """
    this function receives a row from the movies dataframe and extracts its attributes.
    """

    parantheses_open = [pos for pos, char in enumerate(line) if char == '(']
    parantheses_close = [pos for pos, char in enumerate(line) if char == ')']

    if parantheses_open == []:
        "if the movie year is not given, we define it equal to 0"
        return (line ,0)

    movie = line[0 : parantheses_open[-1]-1]
    year = line[parantheses_open[-1] + 1 : parantheses_close[-1]]

    if '-' in year:
        'inside ////'
        years = year.split('-')
        year = [int(item) for item in years]

    else:
        year = int(year)

    return (movie, year)
movies["year"]=0
movies.head()

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0
4,5,Father of the Bride Part II (1995),Comedy,0


In [0]:
# movies preparation 
all_genres = set()
for index, row in movies.iterrows():
    #print(extractor(row['title']))
    genres_list = row['genres'].split('|')
    for gen in genres_list:
        all_genres.add(gen)

In [12]:
all_genres = list(all_genres)
len(all_genres)

20

In [13]:
print(all_genres)
all_genres.index('Drama')

['Horror', 'Romance', '(no genres listed)', 'Film-Noir', 'Documentary', 'Thriller', 'Sci-Fi', 'Children', 'IMAX', 'Animation', 'Action', 'Adventure', 'Mystery', 'War', 'Musical', 'Fantasy', 'Drama', 'Western', 'Comedy', 'Crime']


16

In [14]:
movies_new_columns = ['movieId', 'title', 'year']+all_genres
print(len(movies_new_columns))
print(len(all_genres))
movies_new = pd.DataFrame(columns = movies_new_columns)
print(movies_new)

23
20
Empty DataFrame
Columns: [movieId, title, year, Horror, Romance, (no genres listed), Film-Noir, Documentary, Thriller, Sci-Fi, Children, IMAX, Animation, Action, Adventure, Mystery, War, Musical, Fantasy, Drama, Western, Comedy, Crime]
Index: []


In [0]:
for index, row in movies.iterrows():
  (title, year) = extractor(row['title'])
  
  genres_vector = [0.0]*len(all_genres)
  genres_list = row['genres'].split('|')
  for g in genres_list:
    genres_vector[all_genres.index(g)] = 1.0

  list_to_append = [row['movieId'], title, year] + genres_vector
  dic = {movies_new_columns[i]: list_to_append[i] for i in range(len(list_to_append))} 
  movies_new = movies_new.append(dic, ignore_index=True)

In [16]:
movies_new

,movieId,title,year,Horror,Romance,(no genres listed),Film-Noir,Documentary,Thriller,Sci-Fi,Children,IMAX,Animation,Action,Adventure,Mystery,War,Musical,Fantasy,Drama,Western,Comedy,Crime
0,1,Toy Story,1995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,2,Jumanji,1995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,1995,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,4,Waiting to Exhale,1995,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,5,Father of the Bride Part II,1995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic,2017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
9738,193583,No Game No Life: Zero,2017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
9739,193585,Flint,2017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9740,193587,Bungo Stray Dogs: Dead Apple,2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
#normalise items (movies) vectors 
genres_df = movies_new.iloc[:,3:]
genres_df

,Horror,Romance,(no genres listed),Film-Noir,Documentary,Thriller,Sci-Fi,Children,IMAX,Animation,Action,Adventure,Mystery,War,Musical,Fantasy,Drama,Western,Comedy,Crime
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
9738,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
9739,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9740,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
#normalise genres
genres_df_normalised = genres_df.div(genres_df.sum(axis=1), axis=0)
genres_df_normalised

,Horror,Romance,(no genres listed),Film-Noir,Documentary,Thriller,Sci-Fi,Children,IMAX,Animation,Action,Adventure,Mystery,War,Musical,Fantasy,Drama,Western,Comedy,Crime
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.200000,0.0,0.200000,0.00,0.200000,0.0,0.0,0.0,0.200000,0.000000,0.0,0.200000,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.000000,0.00,0.333333,0.0,0.0,0.0,0.333333,0.000000,0.0,0.000000,0.0
2,0.0,0.500000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.500000,0.0
3,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000000,0.333333,0.0,0.333333,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,1.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.250000,0.25,0.000000,0.0,0.0,0.0,0.250000,0.000000,0.0,0.250000,0.0
9738,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.333333,0.00,0.000000,0.0,0.0,0.0,0.333333,0.000000,0.0,0.333333,0.0
9739,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000000,1.000000,0.0,0.000000,0.0
9740,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.500000,0.50,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0


In [0]:
movies_new = pd.concat([movies_new.iloc[:,0:3], genres_df_normalised], axis=1, sort = False)

In [20]:
#movies dataframe
movies_new

,movieId,title,year,Horror,Romance,(no genres listed),Film-Noir,Documentary,Thriller,Sci-Fi,Children,IMAX,Animation,Action,Adventure,Mystery,War,Musical,Fantasy,Drama,Western,Comedy,Crime
0,1,Toy Story,1995,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.200000,0.0,0.200000,0.00,0.200000,0.0,0.0,0.0,0.200000,0.000000,0.0,0.200000,0.0
1,2,Jumanji,1995,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.000000,0.00,0.333333,0.0,0.0,0.0,0.333333,0.000000,0.0,0.000000,0.0
2,3,Grumpier Old Men,1995,0.0,0.500000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.500000,0.0
3,4,Waiting to Exhale,1995,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000000,0.333333,0.0,0.333333,0.0
4,5,Father of the Bride Part II,1995,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,1.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic,2017,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.250000,0.25,0.000000,0.0,0.0,0.0,0.250000,0.000000,0.0,0.250000,0.0
9738,193583,No Game No Life: Zero,2017,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.333333,0.00,0.000000,0.0,0.0,0.0,0.333333,0.000000,0.0,0.333333,0.0
9739,193585,Flint,2017,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000000,1.000000,0.0,0.000000,0.0
9740,193587,Bungo Stray Dogs: Dead Apple,2018,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.500000,0.50,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0


In [21]:
result = pd.merge(ratings,
                     movies_new[movies_new_columns],
                     on='movieId')
result.head()

,userId,movieId,rating,timestamp,title,year,Horror,Romance,(no genres listed),Film-Noir,Documentary,Thriller,Sci-Fi,Children,IMAX,Animation,Action,Adventure,Mystery,War,Musical,Fantasy,Drama,Western,Comedy,Crime
0,1,1,4.0,964982703,Toy Story,1995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.2,0.0,0.2,0.0,0.0,0.0,0.2,0.0,0.0,0.2,0.0
1,5,1,4.0,847434962,Toy Story,1995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.2,0.0,0.2,0.0,0.0,0.0,0.2,0.0,0.0,0.2,0.0
2,7,1,4.5,1106635946,Toy Story,1995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.2,0.0,0.2,0.0,0.0,0.0,0.2,0.0,0.0,0.2,0.0
3,15,1,2.5,1510577970,Toy Story,1995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.2,0.0,0.2,0.0,0.0,0.0,0.2,0.0,0.0,0.2,0.0
4,17,1,4.5,1305696483,Toy Story,1995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.2,0.0,0.2,0.0,0.0,0.0,0.2,0.0,0.0,0.2,0.0


In [0]:
result = result.drop(['timestamp', 'title'], axis=1)

In [23]:
result

,userId,movieId,rating,year,Horror,Romance,(no genres listed),Film-Noir,Documentary,Thriller,Sci-Fi,Children,IMAX,Animation,Action,Adventure,Mystery,War,Musical,Fantasy,Drama,Western,Comedy,Crime
0,1,1,4.0,1995,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.2,0.0,0.2,0.000000,0.2,0.0,0.0,0.0,0.2,0.000000,0.0,0.2,0.000000
1,5,1,4.0,1995,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.2,0.0,0.2,0.000000,0.2,0.0,0.0,0.0,0.2,0.000000,0.0,0.2,0.000000
2,7,1,4.5,1995,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.2,0.0,0.2,0.000000,0.2,0.0,0.0,0.0,0.2,0.000000,0.0,0.2,0.000000
3,15,1,2.5,1995,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.2,0.0,0.2,0.000000,0.2,0.0,0.0,0.0,0.2,0.000000,0.0,0.2,0.000000
4,17,1,4.5,1995,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.2,0.0,0.2,0.000000,0.2,0.0,0.0,0.0,0.2,0.000000,0.0,0.2,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,610,160341,2.5,1997,0.0,0.0,0.0,0.0,0.0,0.500000,0.0,0.0,0.0,0.0,0.500000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
100832,610,160527,4.5,1971,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.333333
100833,610,160836,3.0,2005,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.000000
100834,610,163937,3.5,2016,0.5,0.0,0.0,0.0,0.0,0.500000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000


In [24]:
# users data frame
users_columns = ['userId']+all_genres
users_new = pd.DataFrame(columns = users_columns)
print(users_new)

Empty DataFrame
Columns: [userId, Horror, Romance, (no genres listed), Film-Noir, Documentary, Thriller, Sci-Fi, Children, IMAX, Animation, Action, Adventure, Mystery, War, Musical, Fantasy, Drama, Western, Comedy, Crime]
Index: []


In [25]:
result.sort_values(['userId', "movieId"], ascending = (True, True))

,userId,movieId,rating,year,Horror,Romance,(no genres listed),Film-Noir,Documentary,Thriller,Sci-Fi,Children,IMAX,Animation,Action,Adventure,Mystery,War,Musical,Fantasy,Drama,Western,Comedy,Crime
0,1,1,4.0,1995,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.2,0.0,0.2,0.000000,0.2,0.000000,0.0,0.0,0.2,0.000000,0.0,0.2,0.000000
215,1,3,4.0,1995,0.000000,0.5,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.5,0.000000
267,1,6,4.0,1995,0.000000,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.333333,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.333333
369,1,47,5.0,1995,0.000000,0.0,0.0,0.0,0.0,0.500000,0.0,0.0,0.0,0.0,0.000000,0.0,0.500000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
572,1,50,5.0,1995,0.000000,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.000000,0.0,0.333333,0.0,0.0,0.0,0.000000,0.0,0.0,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81464,610,166534,4.0,2017,0.333333,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.333333,0.0,0.0,0.000000
68922,610,168248,5.0,2017,0.000000,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.333333,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.333333
81479,610,168250,5.0,2017,1.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
57554,610,168252,5.0,2017,0.000000,0.0,0.0,0.0,0.0,0.000000,0.5,0.0,0.0,0.0,0.500000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000


In [26]:
result[result['userId'] == 2]

,userId,movieId,rating,year,Horror,Romance,(no genres listed),Film-Noir,Documentary,Thriller,Sci-Fi,Children,IMAX,Animation,Action,Adventure,Mystery,War,Musical,Fantasy,Drama,Western,Comedy,Crime
2267,2,333,4.0,1995,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,1.000000,0.000000
15657,2,3578,4.0,2000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.333333,0.333333,0.000000,0.000000,0.0,0.0,0.333333,0.000000,0.000000,0.000000
16296,2,318,3.0,1994,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.500000,0.000000,0.000000,0.500000
16613,2,1704,4.5,1997,0.000000,0.5,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.500000,0.000000,0.000000,0.000000
16754,2,6874,4.0,2003,0.000000,0.0,0.0,0.0,0.0,0.333333,0.000000,0.0,0.000000,0.0,0.333333,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.333333
16885,2,8798,3.5,2004,0.000000,0.0,0.0,0.0,0.0,0.250000,0.000000,0.0,0.000000,0.0,0.250000,0.000000,0.000000,0.000000,0.0,0.0,0.250000,0.000000,0.000000,0.250000
16929,2,46970,4.0,2006,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.500000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.500000,0.000000
16957,2,48516,4.0,2006,0.000000,0.0,0.0,0.0,0.0,0.333333,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.333333,0.000000,0.000000,0.333333
17064,2,58559,4.5,2008,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.250000,0.0,0.250000,0.000000,0.000000,0.000000,0.0,0.0,0.250000,0.000000,0.000000,0.250000
17213,2,60756,5.0,2008,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,1.000000,0.000000


In [0]:
for user_id in range(1,611,1):
  #print(user_id)
  user = result[result['userId'] == user_id]

  new_line = []
  for g in all_genres:
    tempo = user[["rating"]].multiply(user[g], axis="index")
    new_line.append(tempo.sum(axis = 0, skipna = True)[0])

  list_to_append = [user_id] + new_line
  line_lables = ['userId'] + all_genres
  dic = {line_lables[i]: list_to_append[i] for i in range(len(list_to_append))} 
  users_new = users_new.append(dic, ignore_index=True)

Question 8

In [28]:
c=users_new[users_new["userId"]==1]
m=movies_new[movies_new["movieId"]==1]
print(c)
print(m)

def heuristic(c,m):
  n1=0
  n2=0
  s=0
  for i in all_genres:
    n1+=float(c[i])**2
    n2+=float(m[i])**2
    s+=float(c[i])*float(m[i])
  n1=float(n1)**(1/2)
  n2=float(n2)**(1/2)
  if (float(n1)*float(n2)!=0):
    s=float(s/(n1*n2))
  else :
    s=0
  return float(s)
print(heuristic(c,m))   

   userId  Horror    Romance  ...    Western      Comedy      Crime
0     1.0   29.75  37.916667  ...  10.916667  155.130952  65.697619

[1 rows x 21 columns]
  movieId      title  year  Horror  ...  Drama  Western  Comedy  Crime
0       1  Toy Story  1995     0.0  ...    0.0      0.0     0.2    0.0

[1 rows x 23 columns]
0.6156525750175649


In [0]:
recommandation = []

for userId in range (1,6):
  c=users_new[users_new["userId"]==userId]
  m_heur=[-1,-1,-1]
  m_title=["","",""]
  m_new_heuristic=-1

  for movieId in movies_new["movieId"]:
    m=movies_new[movies_new["movieId"]==movieId]
    m_new_heuristic=heuristic(c,m)

    if (m_new_heuristic>m_heur[0]):
      m_heur[2],m_heur[1]=m_heur[1],m_heur[0]
      m_heur[0]=m_new_heuristic
      m_title[2],m_title[1]=m_title[1],m_title[0]
      m_title[0]=m["title"].to_string()
    elif (m_new_heuristic>m_heur[1]):
      m_heur[2]=m_heur[1]
      m_heur[1]=m_new_heuristic
      m_title[2]=m_title[1]
      m_title[1]=m["title"].to_string()
    elif (m_new_heuristic>m_heur[2]):
      m_heur[2]=m_new_heuristic
      m_title[2]=m["title"].to_string()

  recommandation.append(m_title) 

In [30]:
for userId in range (1,6):

  print("User "+ str(userId) + " recommandations : " + str(recommandation[userId-1]))

User 1 recommandations : ['8597    Dragonheart 2: A New Beginning', '6570    Hunting Party, The', '4005    Flashback']
User 2 recommandations : ['19    Money Train', '118    Bad Boys', '1103    Metro']
User 3 recommandations : ['6789    Rogue', '902    Aliens', '9361    Sharknado 4: The 4th Awakens']
User 4 recommandations : ['64    Kicking and Screaming', '67    Big Bully', "74    Antonia's Line (Antonia)"]
User 5 recommandations : ['1394    Out of Sight', '2903    Nurse Betty', '9106    Confessions of a Dangerous Mind']


Exercice 9:

In [0]:
train_data=movies_new.sample(n=400)

#k-means sur les données centrées et réduites
from sklearn import cluster
kmeans = cluster.KMeans(n_clusters=4)
kmeans.fit(train_data)

#index triés des groupes
idk = np.argsort(kmeans.labels_)
#affichage des observations et leurs groupes
print(pandas.DataFrame(train_data.index[idk],train_data.labels_[idk]))
#distances aux centres de classes des observations
print(kmeans.transform(train_data))